In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder


In [ ]:

# Cargar los datos
hotels = pd.read_csv('hotel_bookings.csv')

# Imputar valores faltantes para variables numéricas y categóricas
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')
num_columns = hotels.select_dtypes(include=['float64', 'int64']).columns
cat_columns = hotels.select_dtypes(include=['object']).columns
hotels[num_columns] = num_imputer.fit_transform(hotels[num_columns])
hotels[cat_columns] = cat_imputer.fit_transform(hotels[cat_columns])

# Codificar variables categóricas
label_encoder = LabelEncoder()
for column in cat_columns:
    hotels[column] = label_encoder.fit_transform(hotels[column])

# Ingeniería de características
hotels['total_stay'] = hotels['no_of_weekend_nights'] + hotels['no_of_week_nights']
hotels['weekend_proportion'] = hotels['no_of_weekend_nights'] / hotels['total_stay']
hotels['weekend_proportion'].fillna(0, inplace=True)


In [ ]:

# Dividir los datos en características y variable objetivo
X = hotels.drop('booking_status', axis=1)
y = hotels['booking_status']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir el grid de hiperparámetros para RandomForest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Crear y entrenar el modelo con GridSearchCV
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Mejores parámetros y puntuación
best_params = grid_search.best_params_
best_score = grid_search.best_score_
best_params, best_score
